In [ ]:
! pip install langchain langchain_community weaviate-client 

In [2]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores import Chroma

from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.llms import HuggingFaceHub
import torch
from transformers import ( AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline, )
from langchain import HuggingFacePipeline

from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.document_loaders import TextLoader
import os
import weaviate

#app = Flask(__name__)

#i need to get directory loading working
loader = TextLoader("/home/jonot480/Documents/paulgraham.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=100,chunk_overlap=0)
docs = text_splitter.split_documents(documents)

client = weaviate.Client("http://localhost:8080")
#client = weaviate.connect_to_local()#spin up the docker container
questions = client.collections.create(
        name="CollectionTest",
        vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_transformers(),  # If set to "none" you must always provide vectors yourself. Could be any other "text2vec-*" also.
        generative_config=wvc.config.Configure.Generative.mistral()  # Ensure the `generative-openai` module is used for generative queries
    )

vectorstore=Weaviate(client,"CollectionTest","content") 

#need to get collection queries without re-embedding everytime
weav = Weaviate.from_documents(
    docs, 
    embedding=OllamaEmbeddings(model='nomic-embed-text'),
    client=client
    #prefer_grpc=True, 
    #collection_name="my_documents",
)

retriever = weav.as_retriever()
#ensemble is an alt. retriever 

#llm = Ollama(model="jonorca3") #not sure why this one acts different
llm = ChatOllama(model="jonphi3")

after_rag_template="""
    <context>
    {context}
    </context>
    Question: {question}
    """
after_rag_prompt = ChatPromptTemplate.from_template(after_rag_template)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/search', methods=['POST'])
def search():
    query = request.form['query']
    search_type = request.form['search_type']
    
    if search_type == 'keyword':
        # Keyword search        
        found_docs = weav.similarity_search_by_text(query)                
        after_rag_chain=(
            {"context":retriever,"question":RunnablePassthrough()}
            | after_rag_prompt
            | llm
            | StrOutputParser()    
        )
        results = after_rag_chain.invoke(query)
    elif search_type == 'vector':
        # Vector search
        found_docs = weav.similarity_search_by_vector(query)
        after_rag_chain=(
            {"context":RunnableLambda(lambda x: found_docs),"question":RunnablePassthrough()}
            | after_rag_prompt
            | llm
            | StrOutputParser()    
        )
        results = after_rag_chain.invoke(query)
    elif search_type == 'summarization':
        # Summarization search        
        found_docs = weav.similarity_search(query)
        chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
        results = chain.run(found_docs)
    return render_template('results.html', results=results)

if __name__ == '__main__':
    app.run()

/home/jonot480/Code/Python/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Created a chunk of size 472, which is longer than the specified 100
Created a chunk of size 440, which is longer than the specified 100
Created a chunk of size 243, which is longer than the specified 100
Created a chunk of size 235, which is longer than the specified 100
Created a chunk of size 518, which is longer than the specified 100
Created a chunk of size 408, which is longer than the specified 100
Created a chunk of size 1198, which is longer than the specified 100
Created a chunk of size 488, which is longer than the specified 100
Created a chunk of size 461, which is longer than the specified 100
Created a chunk of size 416, which is longer than the specified 100
Created a chunk of size 218, which is longer than th

ConnectionError: HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /v1/.well-known/openid-configuration (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fa941297790>: Failed to establish a new connection: [Errno 111] Connection refused'))